## To do

### - Merge do dataset de AQI com o dataset de meteorologia ✅

### - Visualização de dados (gráficos, box plots, matriz de correlação, outliers)
### - Teste de hipóteses
### - Feature Engineering

### - Prever com algoritmos o AQI baseado em dados meteorológicos

## Se possível

### - Interpolação noutras coordenadas
### - Data augmentation

In [1]:
import pandas as pd 

In [3]:
# Read the data again

df = pd.read_csv('datasets/FINALDATASET.csv')

In [3]:
# Find the coordinates to interpolate

# import numpy as np

# Define the coordinates

# coordinates = [
#     [41.5549, -8.4067],
#     [41.653315, -8.58779],
#     [41.650791, -8.43569],
#     [41.449721999999994, -8.296389]
# ]

# Convert the coordinates to a numpy array
# coordinates_array = np.array(coordinates)

# Calculate the mean of the latitude and longitude
# mean_latitude = np.mean(coordinates_array[:, 0])
# mean_longitude = np.mean(coordinates_array[:, 1])

# Print the medium point
# print("Medium Point Latitude:", mean_latitude)
# print("Medium Point Longitude:", mean_longitude)

# 41.57718199999999,-8.43164225 - ZONA NOVA ARCADA
# https://www.google.com/maps/place/41%C2%B034'37.9%22N+8%C2%B025'53.9%22W/@41.5772226,-8.4318715,53m/data=!3m1!1e3!4m4!3m3!8m2!3d41.577182!4d-8.4316422

,date,latitude,longitude,parameter,value,tempmax,tempmin,temp,feelslike,dew,...,precip,snow,windspeed,sealevelpressure,cloudcover,visibility,solarradiation,solarenergy,uvindex,AQI
0,2020-12-18,41.5549,-8.4067,pm10,8.39,55.4,50.2,52.8,52.8,48.2,...,0.320,0,16.9,1018.9,93.9,5.3,51.9,4.5,3,None
1,2020-12-18,41.5549,-8.4067,pm25,4.03,55.4,50.2,52.8,52.8,48.2,...,0.320,0,16.9,1018.9,93.9,5.3,51.9,4.5,3,None
2,2021-01-13,41.5549,-8.4067,pm10,83.13,49.7,32.5,40.1,37.5,35.4,...,0.000,0,7.7,1031.8,1.2,6.3,36.1,3.0,2,None
3,2021-01-13,41.5549,-8.4067,pm25,52.27,49.7,32.5,40.1,37.5,35.4,...,0.000,0,7.7,1031.8,1.2,6.3,36.1,3.0,2,None
4,2021-01-21,41.5549,-8.4067,pm10,29.93,58.0,52.0,55.4,55.4,53.3,...,0.518,0,23.1,1009.4,91.0,4.7,38.9,3.3,2,None


In [5]:
# from scipy.interpolate import Rbf
# 
# filtered_df = df[df['parameter'].isin(['pm1', 'pm25', 'pm10','no2'])]
# 
# coordinates = filtered_df[['latitude', 'longitude']].values
# values = filtered_df['value'].values

# Perform IDW interpolation
# rbf = Rbf(coordinates[:, 0], coordinates[:, 1], values)
# interpolated_value = rbf(41.577929249999995, -8.40629225)

# Print the interpolated value
# print("Interpolated Value:", interpolated_value)

In [12]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Carregar os dados
df = pd.read_csv('datasets/FINALDATASET.csv')

# Remover colunas irrelevantes
df.drop(["date", "time", "longitude", "latitude"], axis=1, inplace=True)

# Preencher valores ausentes (se houver)
df.fillna(method='ffill', inplace=True)

# Separar variáveis de entrada (X) e variável de saída (y)
X = df.drop("AQI", axis=1)
y = df["AQI"]

# Divisão dos dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Modelagem e treinamento
model = LinearRegression()
model.fit(X_train, y_train)

# Avaliação do modelo
y_pred_train = model.predict(X_train)
train_rmse = mean_squared_error(y_train, y_pred_train, squared=False)
print("RMSE (conjunto de treinamento):", train_rmse)

y_pred_test = model.predict(X_test)
test_rmse = mean_squared_error(y_test, y_pred_test, squared=False)
print("RMSE (conjunto de teste):", test_rmse)


ValueError: Input X contains NaN.
LinearRegression does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Carregar os dados
df = pd.read_csv('datasets/FINALDATASET.csv')

# Remover colunas irrelevantes
df.drop(["date", "time", "longitude", "latitude"], axis=1, inplace=True)

# Preencher valores ausentes
df.fillna(method='ffill', inplace=True)

# Separar variáveis de entrada (X) e variável de saída (y)
X = df.drop("AQI", axis=1)
y = df["AQI"]

# Divisão dos dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Modelagem e treinamento
model = LinearRegression()
model.fit(X_train, y_train)

# Avaliação do modelo
y_pred_train = model.predict(X_train)
train_rmse = mean_squared_error(y_train, y_pred_train, squared=False)
print("RMSE (conjunto de treinamento):", train_rmse)

y_pred_test = model.predict(X_test)
test_rmse = mean_squared_error(y_test, y_pred_test, squared=False)
print("RMSE (conjunto de teste):", test_rmse)


In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error


df["datetime"] = pd.to_datetime(df["datetime"])  # Converte a coluna "datetime" para o tipo de dados datetime



# Divisão dos dados em conjuntos de treinamento e teste
X = df.drop("AQI", axis=1)
y = df["AQI"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Modelagem e treinamento
model = LinearRegression()
model.fit(X_train, y_train)

# Avaliação do modelo
y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)
mse_train = mean_squared_error(y_train, y_pred_train)
mse_test = mean_squared_error(y_test, y_pred_test)

print("MSE (conjunto de treinamento):", mse_train)
print("MSE (conjunto de teste):", mse_test)


KeyError: 'datetime'